## Best practices for releases

This notebook is a walkthrough of the Cytoscape Release build process. Use the BUILD_PARENT_DIR variable to specify a directory to build cytoscape in.

#### NOTE: Make sure BUILD_PARENT_DIR is in `.gitignore`

1. Update versions
1. Update Core apps to use new API
1. Build Installers
1. Test on different OSs
1. Deploy subrepos to nexus
1. Post the installers to http://github.com/cytoscape/cytoscape/releases
1. Deploy updated core apps

After release:
1. Merge the branch used to build installers into master and tag the release
1. If master release, update the version in dev branch
1. Update Cytoscape manual via https://github.com/cytoscape/cytoscape-manual
1. Update www.github.com/cytoscape/cytoscape.github.com (downloads page, roadmap, etc.)
1. Update system requirements script at https://github.com/cytoscape/cytoscape-admin-scripts/tree/master/system-checker


In [1]:
from subprocess import Popen, PIPE
import os

In [2]:
# Point to build location (the directory to clone parent cytoscape into)
BUILD_PARENT_DIR = os.path.join(os.getcwd(), 'build')
if not os.path.exists(BUILD_PARENT_DIR):
    os.mkdir(BUILD_PARENT_DIR)

os.chdir(BUILD_PARENT_DIR)
![[ -d cytoscape ]] || git clone https://github.com/cytoscape/cytoscape
CYTOSCAPE_ROOT_DIR = os.path.join(BUILD_PARENT_DIR, 'cytoscape')
CYTOSCAPE_DIR = os.path.join(CYTOSCAPE_ROOT_DIR, 'cytoscape')

def cd(directory=BUILD_PARENT_DIR, *subdirs):
    if subdirs:
        directory = os.path.join(directory, *subdirs)
    if os.getcwd() != directory:
        os.chdir(directory)

Cloning into 'cytoscape'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 530 (delta 0), reused 1 (delta 0), pack-reused 529
Receiving objects: 100% (530/530), 142.35 KiB | 0 bytes/s, done.
Resolving deltas: 100% (253/253), done.


In [3]:
STARTING_BRANCH = 'release/3.7.1'     # develop for major release, release/3.X.X for minor release
NEW_BRANCH = 'release/3.7.2'    # branch to create that will be deployed
OLD_VERSION = '3.7.1'  # Version to replace in maven files
NEW_VERSION = '3.7.2'           # New version to set across maven
OLD_RELEASE_VERSION = '3.7.1'   # Release Version to replace on website/version checker

## 1. Pull the develop branch of Cytoscape

In [4]:
cd(CYTOSCAPE_ROOT_DIR)
![[ -d cytoscape ]] || ./cy.sh init
cd(CYTOSCAPE_DIR)
!./cy.sh run-all "git checkout {STARTING_BRANCH}"

Target directory = 
Cytoscape project will be cloned to: /home/cybuilder/cytoscape-admin-scripts/build/cytoscape
Cloning into 'cytoscape'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 530 (delta 0), reused 1 (delta 0), pack-reused 529
Receiving objects: 100% (530/530), 142.35 KiB | 0 bytes/s, done.
Resolving deltas: 100% (253/253), done.
Cloning: parent (URI = git@github.com:cytoscape/cytoscape-parent.git)
Cloning into 'parent'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1601 (delta 16), reused 16 (delta 1), pack-reused 1569
Receiving objects: 100% (1601/1601), 251.22 KiB | 0 bytes/s, done.
Resolving deltas: 100% (601/601), done.
~/cytoscape-admin-scripts/build/cytoscape/cytoscape/parent ~/cytoscape-admin-scripts/build/cytoscape/cytoscape
Already on 'develop'
~/cytoscape-admin-scripts/build/cytoscape/cytoscape
Cloning: api (URI 

## 1a. Reset

In [5]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git clean -f -d'
!./cy.sh run-all 'git reset --hard'

------------------------------------------------------------------------
Executing command: git clean -f -d
--in .
------------------------------------------------------------------------
--in parent
------------------------------------------------------------------------
--in api
------------------------------------------------------------------------
--in impl
------------------------------------------------------------------------
--in support
------------------------------------------------------------------------
--in gui-distribution
------------------------------------------------------------------------
--in app-developer
------------------------------------------------------------------------
------------------------------------------------------------------------
Executing command: git reset --hard
--in .
HEAD is now at a3e5e65 Updated run-all command
------------------------------------------------------------------------
--in parent
HEAD is now at 69b7444 Switch\ to\ 3.7.1


## 2. Create new branch locally

In [6]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git checkout -b {NEW_BRANCH}'
# !./cy.sh run-all 'git pull' # only necessary if branch is pushed

------------------------------------------------------------------------
Executing command: git checkout -b release/3.7.2
--in .
Switched to a new branch 'release/3.7.2'
------------------------------------------------------------------------
--in parent
Switched to a new branch 'release/3.7.2'
------------------------------------------------------------------------
--in api
Switched to a new branch 'release/3.7.2'
------------------------------------------------------------------------
--in impl
Switched to a new branch 'release/3.7.2'
------------------------------------------------------------------------
--in support
Switched to a new branch 'release/3.7.2'
------------------------------------------------------------------------
--in gui-distribution
Switched to a new branch 'release/3.7.2'
------------------------------------------------------------------------
--in app-developer
Switched to a new branch 'release/3.7.2'
-------------------------------------------------------------

In [7]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git status'

------------------------------------------------------------------------
Executing command: git status
--in .
# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
--in parent
# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
--in api
# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
--in impl
# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
--in support
# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
--in gui-distribution
# On branch release/3.7.2
nothing to commit, working directory clean
---------------------------------------------

## 3. Update versions in pom files and all documents

In [8]:
cd(CYTOSCAPE_DIR)
!mvn versions:set -DnewVersion={NEW_VERSION}

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for org.cytoscape:app-api:bundle:3.7.1
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.cytoscape:jobs-api:jar -> duplicate declaration of version ${project.version} @ line 180, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] ------------------------------------------------------------------------
[INFO] Reactor Build Order:
[INFO] 
[INFO] Basic Cytoscape POM Configuration
[INFO] Cytoscape API Parent Project POM
[INFO] Cytoscape Documentation Taglets
[INFO] Cytoscape Event API (event-api)
[INFO] Cytoscape Model API (model-api)
[INFO] Cytoscape Service API (service-api)
[INFO] Cytoscape View Model API (viewmodel-a

[INFO] Final Memory: 35M/429M
[INFO] ------------------------------------------------------------------------


In [9]:
# then update the versions in the parent directory
cd(CYTOSCAPE_DIR, 'parent')
!mvn versions:set -DnewVersion={NEW_VERSION}

[INFO] Scanning for projects...
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Basic Cytoscape POM Configuration 3.7.1
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:set (default-cli) @ parent ---
[INFO] Searching for local aggregator root...
[INFO] Local aggregation root: /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape
[INFO] Processing change of org.cytoscape:parent:3.7.1 -> 3.7.2
[INFO] Processing org.cytoscape:parent
[INFO]     Updating project org.cytoscape:parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape.app-developer:cytoscape
[INFO]     Updating parent org.cytoscape:parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO]     Updating project org.cytoscape.app-developer:cytoscape
[INFO]         from version 3.7.1 to 

[INFO] 
[INFO] Processing org.cytoscape:filter-impl
[INFO]     Updating parent org.cytoscape:impl-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO]     Updating project org.cytoscape:filter-impl
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:filter2-impl
[INFO]     Updating parent org.cytoscape:impl-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:graphml-impl
[INFO]     Updating parent org.cytoscape:impl-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:group-api
[INFO]     Updating parent org.cytoscape:api-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:group-impl
[INFO]     Updating parent org.cytoscape:impl-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO]     Updating project org.cytoscape:group-impl
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:gui-cmdline-parser-impl
[INFO

[INFO] 
[INFO] Processing org.cytoscape:task-testing-impl
[INFO]     Updating parent org.cytoscape:support-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO]     Updating project org.cytoscape:task-testing-impl
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:util-api
[INFO]     Updating parent org.cytoscape:api-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:util-impl
[INFO]     Updating parent org.cytoscape:impl-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO]     Updating project org.cytoscape:util-impl
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:viewmodel-api
[INFO]     Updating parent org.cytoscape:api-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO] 
[INFO] Processing org.cytoscape:viewmodel-impl-parent
[INFO]     Updating parent org.cytoscape:impl-parent
[INFO]         from version 3.7.1 to 3.7.2
[INFO]     Updating project org.cytoscape:viewmo

## 3a. Commit (or reset) versions in pom files

In [10]:
cd(CYTOSCAPE_DIR)
#!mvn version:revert
!mvn versions:commit

cd(CYTOSCAPE_DIR, 'parent')
#!mvn version:revert
!mvn versions:commit

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for org.cytoscape:app-api:bundle:3.7.2
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.cytoscape:jobs-api:jar -> duplicate declaration of version ${project.version} @ line 180, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] ------------------------------------------------------------------------
[INFO] Reactor Build Order:
[INFO] 
[INFO] Basic Cytoscape POM Configuration
[INFO] Cytoscape API Parent Project POM
[INFO] Cytoscape Documentation Taglets
[INFO] Cytoscape Event API (event-api)
[INFO] Cytoscape Model API (model-api)
[INFO] Cytoscape Service API (service-api)
[INFO] Cytoscape View Model API (viewmodel-a

[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/event-api/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Model API (model-api) 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ model-api ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/model-api/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Service API (service-api) 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ service-api ---
[I

[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ app-api ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/app-api/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Work Swing API (work-swing-api) 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ work-swing-api ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/work-swing-api/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Swing Utility API (swing-util-api) 3.7.2
[INFO] ----------------------------------------------------

[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/impl/graphml-impl/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape IO Impl Performance (io-impl-performance) 3.7.2
[INFO] ------------------------------------------------------------------------
[WARNING] The POM for org.eclipse.m2e:lifecycle-mapping:jar:1.0.0 is missing, no dependency information available
[WARNING] Failed to retrieve plugin descriptor for org.eclipse.m2e:lifecycle-mapping:1.0.0: Plugin org.eclipse.m2e:lifecycle-mapping:1.0.0 or one of its dependencies could not be resolved: Failed to read artifact descriptor for org.eclipse.m2e:lifecycle-mapping:jar:1.0.0
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ io-impl-performance ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytosc

[WARNING] Failed to retrieve plugin descriptor for org.eclipse.m2e:lifecycle-mapping:1.0.0: Plugin org.eclipse.m2e:lifecycle-mapping:1.0.0 or one of its dependencies could not be resolved: Failed to read artifact descriptor for org.eclipse.m2e:lifecycle-mapping:jar:1.0.0
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ presentation-impl ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/impl/presentation-impl/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Property Impl (property-impl) 3.7.2
[INFO] ------------------------------------------------------------------------
[WARNING] The POM for org.eclipse.m2e:lifecycle-mapping:jar:1.0.0 is missing, no dependency information available
[WARNING] Failed to retrieve plugin descriptor for org.eclipse.m2e:lifecycle-mapping:1.0.0: P

[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/impl/application-impl/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Datasource Impl (datasource-impl) 3.7.2
[INFO] ------------------------------------------------------------------------
[WARNING] The POM for org.eclipse.m2e:lifecycle-mapping:jar:1.0.0 is missing, no dependency information available
[WARNING] Failed to retrieve plugin descriptor for org.eclipse.m2e:lifecycle-mapping:1.0.0: Plugin org.eclipse.m2e:lifecycle-mapping:1.0.0 or one of its dependencies could not be resolved: Failed to read artifact descriptor for org.eclipse.m2e:lifecycle-mapping:jar:1.0.0
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ datasource-impl ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cyto

[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/support/archetypes/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape 3 Archetype - Creates an App providing public API 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ api-provider-app ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/support/archetypes/api-provider-app/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape 3 Archetype - Creates an App with a Task 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[

[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ application-metadata-impl ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/gui-distribution/application-metadata-impl/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape GUI-Distribution Integration Test (integration-test) 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ integration-test ---
[INFO] Accepting all changes to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/gui-distribution/integration-test/pom.xml
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Feature Repos

[INFO] Final Memory: 49M/695M
[INFO] ------------------------------------------------------------------------
[INFO] Scanning for projects...
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Basic Cytoscape POM Configuration 3.7.2
[INFO] ------------------------------------------------------------------------
[INFO] 
[INFO] --- versions-maven-plugin:2.7:commit (default-cli) @ parent ---
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time: 0.736s
[INFO] Finished at: Fri Aug 23 10:33:50 PDT 2019
[INFO] Final Memory: 15M/303M
[INFO] ------------------------------------------------------------------------


## 3b. Update version in other files (poms, executables)

In [11]:
%%bash -s "$OLD_VERSION" "$NEW_VERSION" "$CYTOSCAPE_DIR"

cd "$3"

echo "* Updating version numbers in pom files..."
find . -name pom.xml | xargs sed -i "" -e "s/$1/$2/g"
echo "Done"

echo "* Updating version numbers in executables..."
ls gui-distribution/assembly/src/main/bin/cytoscape.* | xargs sed -i "" -e "s/$1/$2/g"
echo "Done"

* Updating version numbers in pom files...
Done
* Updating version numbers in executables...
Done


sed: can't read : No such file or directory
sed: can't read : No such file or directory


In [12]:
# HelpTaskFactory uses hard coded versioning.
cd(CYTOSCAPE_DIR, 'impl', 'core-task-impl', 'src', 'main', 'java', 'org', 
   'cytoscape', 'task', 'internal', 'help')
!echo "* Updating Manual link"
!ls HelpTaskFactory.jav* | xargs sed -i "" -e "s/{OLD_RELEASE_VERSION}/{NEW_VERSION}/g"

* Updating Manual link
sed: can't read : No such file or directory


## 3c. Maven clean and check for outstanding occurrences of old version

In [13]:
!echo "* Cleaning old installers"
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
!mvn clean -q
cd(CYTOSCAPE_DIR, 'gui-distribution', 'splash-launcher')
!mvn clean -q
!echo "Done"

* Cleaning old installers
Done


In [14]:
cd(CYTOSCAPE_DIR)
ESCAPED = OLD_VERSION.replace('.', '\\.')
!echo "* Showing occurrences of {ESCAPED} in `pwd`:" && grep -ri '{ESCAPED}' .

* Showing occurrences of 3\.7\.1 in /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape:
./.git/config:[branch "release/3.7.1"]
./.git/config:	merge = refs/heads/release/3.7.1
./.git/packed-refs:a3e5e65497a0e868429146df1a709ab5595f7f1f refs/remotes/origin/release/3.7.1
./.git/packed-refs:a3e5e65497a0e868429146df1a709ab5595f7f1f refs/tags/3.7.1
./.git/logs/refs/heads/release/3.7.1:0000000000000000000000000000000000000000 a3e5e65497a0e868429146df1a709ab5595f7f1f David Otasek <dotasek.dev@gmail.com> 1566581573 -0700	branch: Created from refs/remotes/origin/release/3.7.1
./.git/logs/HEAD:8d3c88ce7ba8d1daae921fe1c144dbdd2c7e94ee a3e5e65497a0e868429146df1a709ab5595f7f1f David Otasek <dotasek.dev@gmail.com> 1566581573 -0700	checkout: moving from develop to release/3.7.1
./.git/logs/HEAD:a3e5e65497a0e868429146df1a709ab5595f7f1f a3e5e65497a0e868429146df1a709ab5595f7f1f David Otasek <dotasek.dev@gmail.com> 1566581588 -0700	checkout: moving from release/3.7.1 to release/3.7.2
./pare

./gui-distribution/.git/packed-refs:4ce724269eb557cd494da8af2b3c019d6a3714ba refs/remotes/origin/release/3.7.1
./gui-distribution/.git/packed-refs:7c98d2c2a2d760f464566b7e66201a480c6cd3c6 refs/tags/3.7.1
./gui-distribution/.git/logs/refs/heads/release/3.7.1:0000000000000000000000000000000000000000 4ce724269eb557cd494da8af2b3c019d6a3714ba David Otasek <dotasek.dev@gmail.com> 1566581573 -0700	branch: Created from refs/remotes/origin/release/3.7.1
./gui-distribution/.git/logs/HEAD:2f397feda884b109e7dab04fad4d60a8d12252bd 4ce724269eb557cd494da8af2b3c019d6a3714ba David Otasek <dotasek.dev@gmail.com> 1566581573 -0700	checkout: moving from develop to release/3.7.1
./gui-distribution/.git/logs/HEAD:4ce724269eb557cd494da8af2b3c019d6a3714ba 4ce724269eb557cd494da8af2b3c019d6a3714ba David Otasek <dotasek.dev@gmail.com> 1566581588 -0700	checkout: moving from release/3.7.1 to release/3.7.2
./app-developer/.git/config:[branch "release/3.7.1"]
./app-developer/.git/config:	merge = refs/heads/release/3.

In [15]:
cd(CYTOSCAPE_DIR)
ESCAPED = OLD_RELEASE_VERSION.replace('.', '\\.')
!echo "* Showing occurrences of {ESCAPED} in `pwd`:" && grep -ri '{ESCAPED}' .

* Showing occurrences of 3\.7\.1 in /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape:
./.git/config:[branch "release/3.7.1"]
./.git/config:	merge = refs/heads/release/3.7.1
./.git/packed-refs:a3e5e65497a0e868429146df1a709ab5595f7f1f refs/remotes/origin/release/3.7.1
./.git/packed-refs:a3e5e65497a0e868429146df1a709ab5595f7f1f refs/tags/3.7.1
./.git/logs/refs/heads/release/3.7.1:0000000000000000000000000000000000000000 a3e5e65497a0e868429146df1a709ab5595f7f1f David Otasek <dotasek.dev@gmail.com> 1566581573 -0700	branch: Created from refs/remotes/origin/release/3.7.1
./.git/logs/HEAD:8d3c88ce7ba8d1daae921fe1c144dbdd2c7e94ee a3e5e65497a0e868429146df1a709ab5595f7f1f David Otasek <dotasek.dev@gmail.com> 1566581573 -0700	checkout: moving from develop to release/3.7.1
./.git/logs/HEAD:a3e5e65497a0e868429146df1a709ab5595f7f1f a3e5e65497a0e868429146df1a709ab5595f7f1f David Otasek <dotasek.dev@gmail.com> 1566581588 -0700	checkout: moving from release/3.7.1 to release/3.7.2
./pare

## 4. Build Cytoscape and ensure no errors
This may take a while. Expect to build subrepos first before building from the root directory

In [21]:
cd(CYTOSCAPE_DIR)
with open('build_output.txt', 'w') as outf:
    process = Popen('mvn -fae clean install -Dmaven.test.skip=true'.split(' '), 
                stdout=outf,
                cwd=CYTOSCAPE_DIR)
    process.wait()

print("Showing ERROR lines in build...")
!cat build_output.txt | grep ERROR

Showing ERROR lines in build...


## 5. Commit changes to new branch

In [18]:
# if successful, commit the changes
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git add .'
!./cy.sh run-all 'git commit -m "Update version to {NEW_VERSION}"'

------------------------------------------------------------------------
Executing command: git add .
--in .
------------------------------------------------------------------------
--in parent
error: open("sedv6akEx"): Permission denied
error: unable to index file sedv6akEx
fatal: adding files failed
------------------------------------------------------------------------
--in api
------------------------------------------------------------------------
--in impl
------------------------------------------------------------------------
--in support
------------------------------------------------------------------------
--in gui-distribution
------------------------------------------------------------------------
--in app-developer
------------------------------------------------------------------------
------------------------------------------------------------------------
Executing command: git commit -m "Update version to 3.7.2"
--in .
[release/3.7.2 72df44e] Update version to 3.7.2

In [19]:
cd(CYTOSCAPE_DIR)
!./cy.sh status

------------------------------------------------------------------------
- .:

# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
- parent:

# On branch release/3.7.2
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   pom.xml
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	sedv6akEx
no changes added to commit (use "git add" and/or "git commit -a")
------------------------------------------------------------------------
- api:

# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
- impl:

# On branch release/3.7.2
nothing to commit, working directory clean
------------------------------------------------------------------------
- 

## 6. Build Cytoscape installers
This requires Install4J to be configured on your machine via the instructions at the Cytoscape README

In [26]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
!mvn clean install -U

[INFO] Scanning for projects...
[INFO]                                                                         
[INFO] ------------------------------------------------------------------------
[INFO] Building Cytoscape Release Packaging 3.7.2
[INFO] ------------------------------------------------------------------------
Downloading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/distribution/cytoscape/3.7.2/cytoscape-3.7.2.pom
Downloading: http://code.cytoscape.org/nexus/content/repositories/thirdparty/org/cytoscape/distribution/cytoscape/3.7.2/cytoscape-3.7.2.pom
Downloading: http://code.cytoscape.org/nexus/content/repositories/ops4j-release/org/cytoscape/distribution/cytoscape/3.7.2/cytoscape-3.7.2.pom
Downloading: http://code.cytoscape.org/nexus/content/repositories/spring-osgi/org/cytoscape/distribution/cytoscape/3.7.2/cytoscape-3.7.2.pom
Downloading: http://code.cytoscape.org/nexus/content/repositories/spring-osgi-external/org/cytoscape/distribution/cy

- Update core apps to rely on deployed releases, test
- Deploy and push them to app store
- Merge develop into master branch
- Tag and release master, publish release
- Change dev branch version

## 7. Deploy JARs to Nexus

- Need to have our repository properly configured in ~/.m2/settings.xml
- Cytoscape Project POM will fail. This is expected

In [24]:
cd(CYTOSCAPE_DIR)
!mvn clean deploy

[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for org.cytoscape:app-api:bundle:3.7.2
[WARNING] 'dependencies.dependency.(groupId:artifactId:type:classifier)' must be unique: org.cytoscape:jobs-api:jar -> duplicate declaration of version ${project.version} @ line 180, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] ------------------------------------------------------------------------
[INFO] Reactor Build Order:
[INFO] 
[INFO] Basic Cytoscape POM Configuration
[INFO] Cytoscape API Parent Project POM
[INFO] Cytoscape Documentation Taglets
[INFO] Cytoscape Event API (event-api)
[INFO] Cytoscape Model API (model-api)
[INFO] Cytoscape Service API (service-api)
[INFO] Cytoscape View Model API (viewmodel-a

[INFO] 
[INFO] --- maven-clean-plugin:2.4.1:clean (default-clean) @ api-parent ---
[INFO] Deleting /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/target
[INFO] 
[INFO] >>> maven-source-plugin:2.4:jar (attach-sources) @ api-parent >>>
[INFO] 
[INFO] <<< maven-source-plugin:2.4:jar (attach-sources) @ api-parent <<<
[INFO] 
[INFO] --- maven-source-plugin:2.4:jar (attach-sources) @ api-parent ---
[INFO] 
[INFO] >>> maven-source-plugin:2.4:aggregate (aggregate-source) @ api-parent >>>
[INFO]                                                                         
[INFO] >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[INFO] Forking Cytoscape Property API (property-api) 3.7.2
[INFO] >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[INFO] 
[INFO] --- maven-jaxb2-plugin:0.8.1:generate (bookmarks) @ property-api ---
[INFO] Started execution.
[INFO] JAXB API is loaded from the [jar:file:/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.

[INFO]                                                                         
[INFO] >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[INFO] Forking Cytoscape Feature Repository 3.7.2
[INFO] >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[INFO] 
[INFO] --- maven-resources-plugin:2.6:copy-resources (copy-resources) @ features ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 1 resource
[INFO] 
[INFO] <<< maven-source-plugin:2.4:aggregate (aggregate-source) @ api-parent <<<
[INFO] 
[INFO] --- maven-source-plugin:2.4:aggregate (aggregate-source) @ api-parent ---
[INFO] Building jar: /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/target/api-parent-3.7.2-sources.jar
[INFO] 
[INFO] --- maven-install-plugin:2.3.1:install (default-install) @ api-parent ---
[INFO] Installing /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/pom.xml to /home/cybuilder/.m2/repository/org/cytos

[INFO] debug:false
[INFO] args:[]
[INFO] forceRegenerate:false
[INFO] removeOldOutput:false
[INFO] produces:[**/*.*, **/*.java, **/bgm.ser, **/jaxb.properties]
[INFO] otherDepends:null
[INFO] episodeFile:/home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/impl/io-impl/impl/target/generated-sources/xjc/cytables/META-INF/sun-jaxb.episode
[INFO] episode:true
[INFO] classpathElements:[/home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/impl/io-impl/impl/target/classes]
[INFO] plugins:null
[INFO] episodes:null
[INFO] useDependenciesAsEpisodes:false
[INFO] xjcPlugins:null
[INFO] episodes:null
[INFO] schemaFiles (calculated):[/home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/impl/io-impl/impl/src/main/resources/xsd/cytables.xsd]
[INFO] bindingFiles (calculated):[]
[INFO] xjcPluginArtifacts (resolved):[]
[INFO] xjcPluginFiles (resolved):[]
[INFO] xjcPluginURLs (resolved):[]
[INFO] episodeArtifacts (resolved):[]
[INFO] episodeFiles (resolved):[]
[INFO

Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/3.7.2/taglets-3.7.2.jar (12 KB at 19.0 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/3.7.2/taglets-3.7.2.pom
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/3.7.2/taglets-3.7.2.pom (4 KB at 8.0 KB/sec)
Downloading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/maven-metadata.xml
Downloaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/maven-metadata.xml (1001 B at 2.9 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/maven-metadata.xml
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/maven-metadata.xml (2 KB at 2.4 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/taglets/3.7.

...

[INFO] Installing org/cytoscape/ci-api/3.7.2/ci-api-3.7.2.jar
[INFO] Writing OBR metadata
[INFO] 
[INFO] --- maven-deploy-plugin:3.0.0-M1:deploy (default-deploy) @ ci-api ---
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/ci-api/3.7.2/ci-api-3.7.2.jar
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/ci-api/3.7.2/ci-api-3.7.2.jar (6 KB at 11.7 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/ci-api/3.7.2/ci-api-3.7.2.pom
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/ci-api/3.7.2/ci-api-3.7.2.pom (3 KB at 5.8 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/ci-api/3.7.2/ci-api-3.7.2-sources.jar
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/ci-api/3.7.2/ci-api-3.7.2-sources.jar (4 KB at 7.0 KB/sec)
Downloading: http://code.cytoscape.org/nexus/c

[INFO] episodeFile:/home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/property-api/target/generated-sources/xjc/bookmarks/META-INF/sun-jaxb.episode
[INFO] episode:true
[INFO] classpathElements:[/home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/property-api/target/classes]
[INFO] plugins:null
[INFO] episodes:null
[INFO] useDependenciesAsEpisodes:false
[INFO] xjcPlugins:null
[INFO] episodes:null
[INFO] schemaFiles (calculated):[/home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/property-api/src/main/xsd/bookmarks.xsd, /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/property-api/src/main/xsd/xlinks-2001.xsd]
[INFO] bindingFiles (calculated):[]
[INFO] xjcPluginArtifacts (resolved):[]
[INFO] xjcPluginFiles (resolved):[]
[INFO] xjcPluginURLs (resolved):[]
[INFO] episodeArtifacts (resolved):[]
[INFO] episodeFiles (resolved):[]
[INFO] optionsConfiguration:OptionsConfiguration [specVersion=2.1
 generateDirectory=/ho

[INFO] Installing org/cytoscape/work-api/3.7.2/work-api-3.7.2.jar
[INFO] Writing OBR metadata
[INFO] 
[INFO] --- maven-deploy-plugin:3.0.0-M1:deploy (default-deploy) @ work-api ---
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/work-api/3.7.2/work-api-3.7.2.jar
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/work-api/3.7.2/work-api-3.7.2.jar (45 KB at 67.5 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/work-api/3.7.2/work-api-3.7.2.pom
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/work-api/3.7.2/work-api-3.7.2.pom (3 KB at 6.0 KB/sec)
Uploading: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/work-api/3.7.2/work-api-3.7.2-sources.jar
Uploaded: http://code.cytoscape.org/nexus/content/repositories/releases/org/cytoscape/work-api/3.7.2/work-api-3.7.2-sources.jar (71 KB at 95.8 KB/sec)
Uploading: ht


Results :

Tests in error: 
  test2DLexicon(org.cytoscape.view.presentation.BasicVisualLexiconTest): Can't connect to X11 window server using 'localhost:10.0' as the value of the DISPLAY variable.
  testTree(org.cytoscape.view.presentation.BasicVisualLexiconTest): Could not initialize class org.cytoscape.view.presentation.property.BasicVisualLexicon
  testLexiconInstances(org.cytoscape.view.presentation.BasicVisualLexiconTest): Could not initialize class org.cytoscape.view.presentation.property.BasicVisualLexicon
  testGetAllDecendents(org.cytoscape.view.presentation.BasicVisualLexiconTest): Could not initialize class org.cytoscape.view.presentation.property.BasicVisualLexicon
  testIsSupported(org.cytoscape.view.presentation.BasicVisualLexiconTest): Could not initialize class org.cytoscape.view.presentation.property.BasicVisualLexicon
  testGetSupportedValueRange(org.cytoscape.view.presentation.BasicVisualLexiconTest): Could not initialize class org.cytoscape.view.presentation.proper

[INFO] Final Memory: 63M/1173M
[INFO] ------------------------------------------------------------------------
[ERROR] Failed to execute goal org.apache.maven.plugins:maven-surefire-plugin:2.12.4:test (default-test) on project presentation-api: There are test failures.
[ERROR] 
[ERROR] Please refer to /home/cybuilder/cytoscape-admin-scripts/build/cytoscape/cytoscape/api/presentation-api/target/surefire-reports for the individual test results.
[ERROR] -> [Help 1]
[ERROR] 
[ERROR] To see the full stack trace of the errors, re-run Maven with the -e switch.
[ERROR] Re-run Maven using the -X switch to enable full debug logging.
[ERROR] 
[ERROR] For more information about the errors and possible solutions, please read the following articles:
[ERROR] [Help 1] http://cwiki.apache.org/confluence/display/MAVEN/MojoFailureException
[ERROR] 
[ERROR] After correcting the problems, you can resume the build with the command
[ERROR]   mvn <goals> -rf :presentation-api


## 8. Sign DMG and upload release to github

In [ ]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
!./sign-dmg.sh && mv -f target/install4j/signed/*.dmg target/install4j/
!rmdir target/install4j/signed

#### Create a new release with Cytoscape installers: https://github.com/cytoscape/cytoscape/releases/new

## 9. Update Cytoscape.org, manual, and admin-scripts

Publish JavaDocs by building http://code.cytoscape.org/jenkins/job/cytoscape-3-javadoc/

Generate Release Notes from JIRA https://cytoscape.atlassian.net/projects/CYTOSCAPE

In [ ]:
# Update cytoscape.org

cd(BUILD_PARENT_DIR)
![[ -d cytoscape.github.com ]] || git clone http://github.com/cytoscape/cytoscape.github.com

![[ -d cytoscape-manual ]] || git clone http://github.com/cytoscape/cytoscape-manual

### Cytoscape.org changes

#### Roadmap
cytoscape.github.com/roadmap.html
- Remove section from Vision
- Move release dates 6-8 months forward
- Add section to Bottom of page listing features

#### JS Files:
- js/download.js
  - Copy JSON block to previous versions
- js/setup_page.js
  - Change release version
  
#### Welcome Letter
- Use template at cellar/users/bsettle/Documents/Cytoscape/Releases/
- Must update all sections

#### Release Notes
- releasenotes.html
  - Update welcome letter link
  - Update Latest Release Notes and Older Versions list
- release_notes_3_X_X.html
  - Copied from previous release notes
  - Edit HTML sections to add div's about new features
  - Add screenshots

In [ ]:
cd(BUILD_PARENT_DIR, 'cytoscape.github.com')
!git add .
!git commit -m "Updated for release {NEW_VERSION}"

### Manual
Refer to https://github.com/cytoscape/cytoscape-manual/ for manual changes and building

In [ ]:
#Change docs/conf.py
cd(BUILD_PARENT_DIR, 'cytoscape-manual', 'docs')
!ls conf.p* | xargs sed -i "" -e "s/{OLD_RELEASE_VERSION}/{NEW_VERSION}/g"
!git add .
!git commit -m "Updated for {NEW_VERSION} release"
!git push

- Update all sections with new features and screenshots
#### NOTE: ONLY TAG WHEN READY, Tagging posts the manual to ReadTheDocs
#### Only Major version updates are published. Not minor

Webhooks may be discontinued 1/31/2019. Keep an eye out for auto-building ReadTheDocs
To build it manually, you must have access to the ReadTheDocs page:
- https://readthedocs.org/projects/cytoscape-working-copy/

In [ ]:
cd(BUILD_PARENT_DIR, 'cytoscape-manual')
!git tag {NEW_VERSION}

### Update cytoscape-admin-scripts

In [ ]:
#system-checker files are in this repo
cd(BUILD_PARENT_DIR)
!ls system-checker/* | xargs sed -i "" -e "s/{OLD_RELEASE_VERSION}/{NEW_VERSION}/g"
!git add .
!git commit -m "Updated system checker"

### Publish the manual, website, and create a release on github
Check status before pushing!

In [ ]:
# Bump up the system checker scripts
cd(BUILD_PARENT_DIR)
!git status
#!git push

# Tag and publish Manual
cd(BUILD_PARENT_DIR, 'cytoscape-manual')
!git status
#!git push origin {NEW_VERSION}

# Update website
cd(BUILD_PARENT_DIR, 'cytoscape.github.com')
!git status
#!git push

### Update the file at /data/chianti/html/cytoscape-news/news.html

In [ ]:
!curl --silent http://chianti.ucsd.edu/cytoscape-news/news.html | grep "latestVersion"

### Announce the release on app-dev, cytoscape-announce, cytostaff

In [ ]:
updates = [
    'LIST',
    'UPDATES',
    'HERE'
]
update_list = '\n'.join([' - ' + update for update in updates])
welcome_letter = 'http://www.cytoscape.org/cy3_welcome_letter_v20.pdf'
release_notes = 'http://www.cytoscape.org/release_notes_3_7_1.html'
message = '''Cytoscape {version} is released!

=+=+ =+=+ =+=+ =+=+ =+=+ Hello Cytoscapers! =+=+ =+=+ =+=+ =+=+ =+=+
 
We're very pleased and excited to announce availability of Cytoscape {version}, effective immediately!

To get a copy, surf to http://cytoscape.org/download, and download and execute the installer for your workstation.

There are many new and exciting features in {version}, including:
{update_list}

Note that with {version}, Windows & Mac systems can run Java 9 or 10 without conflicting with Cytoscape -- it installs and uses a private Java 8.

For Linux, you must be sure Java 8 is installed and is first on the PATH or is named via the JAVA_HOME environment variable.

If you find any issues with this release, please let us know by posting to cytoscape-helpdesk@googlegroups.com … we very much value your feedback!

Please check the {version} materials in our Welcome Letter ({welcome_letter}) and the Release Notes ({release_notes}).

Tell all your friends ...

We appreciate your checking this out, and we hope you find it as great to work with as we do!

=+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+'''
print(message.format(version=NEW_VERSION, update_list=update_list, 
                     welcome_letter=welcome_letter, release_notes=release_notes))